In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [0]:
from google.colab import drive 
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
pwd

'/content'

In [0]:
cd /content/drive/My Drive/Fire

/content/drive/My Drive/Fire


In [0]:
import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from IPython.display import display
from PIL import Image
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
from skimage import transform
import os
import random
from keras.models import model_from_json
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.models import model_from_json
import cv2
from skimage import transform
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [0]:
FAST_RUN = False
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3 # RGB color

In [0]:
def CNN_Classification():
  model = Sequential()

  model.add(Conv2D(64, (5, 5), strides=(4, 4), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(3, 3),strides=(2, 2) ))
  #model.add(Dropout(0.25))

  model.add(Conv2D(128, (4, 4), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
  #model.add(Dropout(0.25))

  model.add(Conv2D(256, (1, 1), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
  #model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(4090, activation='tanh'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Dense(4090, activation='tanh'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Dense(2, activation='softmax'))
  optimizer = keras.optimizers.sgd(lr = 0.001)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  #Save Model
  CarsClassification_json =model.to_json()
  with open('modelfire2.json', "w") as json_file:
      json_file.write(CarsClassification_json)
      json_file.close()  
  model.summary()
  print(model.summary())
  return model

In [0]:
pos_dir= ["/content/drive/My Drive/Fire/Datasetnew/Fire"]
neg_dir= ["/content/drive/My Drive/Fire/Datasetnew/NoFire"]

for pos_path in pos_dir:
  print(pos_path)
for neg_path in neg_dir:
  print(neg_path)

/content/drive/My Drive/Fire/Datasetnew/Fire
/content/drive/My Drive/Fire/Datasetnew/NoFire


In [0]:
def Load_Image(pos_dir, neg_dir):
    train_images = []
    train_labels = []

    for pos_path in pos_dir:
      Images_Positive=os.listdir(pos_path)
      for image in Images_Positive:
          path=os.path.join(pos_path,image)
          img = cv2.imread(path)
          train_images.append(transform.resize(img,(224,224,3)))
          l = [1,0]
          train_labels.append(l)
    for neg_path in neg_dir:
      Images_Negative=os.listdir(neg_path)
      for image in Images_Negative:
          path=os.path.join(neg_path,image)
          img = cv2.imread(path)
  #       print(path)
          train_images.append(transform.resize(img,(224,224,3)))
          l = [0,1]
          train_labels.append(l)
    np.save("train_image_fire2",train_images)
    np.save("train_label_fire2",train_labels)
    return np.array(train_images), np.array(train_labels)

In [0]:
def Train_Test_split(train_data, train_labels, fraction):
    idx = np.random.permutation(train_data.shape[0])
    index = int(len(train_data)*fraction)
    return train_data[:index], train_labels[:index], train_data[index:], train_labels[index:]

In [0]:
def Train_Model(pos_dir, neg_dir, fraction):
    Train_Image,Lable_Image=Load_Image(pos_dir, neg_dir)
    fraction = 0.95
    train_data, train_labels, test_data, test_labels = Train_Test_split(Train_Image,Lable_Image, fraction)
    print ("Train data size: ", len(train_data))
    print ("Test data size: ", len(test_data))
    CNN=CNN_Classification()
    print ("Train data shape: ", train_data.shape)
    print ("Test data shape: ", test_data.shape)
    idx = np.random.permutation(train_data.shape[0])
    CNN.fit(train_data[idx], train_labels[idx], batch_size = 64, epochs = 30)
    #Save weight
    CNN.save_weights('weighfire2.h5')
    predicted_test_labels = np.argmax(CNN.predict(test_data), axis=1)
    test_labels = np.argmax(test_labels, axis=1)
    score = CNN.evaluate(test_data, test_labels, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print ("Actual test labels:", test_labels)
    print ("Predicted test labels:", predicted_test_labels)
    print ("Accuracy score:", accuracy_score(test_labels, predicted_test_labels))

In [0]:
def Class_object(img,Loaded_Model):
    img_test=[]
    img_test.append(transform.resize(img,(224,224,3)))
    Input_test=np.array(img_test)
    print("Pridict: ",Loaded_Model.predict(Input_test))
    if(Loaded_Model.predict(Input_test)[0][0]>0.7):
        print(1)
        return 1
    else:
        print(0)
        return 0

In [19]:
if __name__ == "__main__":
    pos_dir= ["/content/drive/My Drive/Fire/Datasetnew/Fire"]
    neg_dir= ["/content/drive/My Drive/Fire/Datasetnew/NoFire"]
    fraction= 0.95;
    Train_Image,Lable_Image= Load_Image(pos_dir, neg_dir) 
    Train_Image= np.load("train_image_fire2.npy")
    Lable_Image= np.load("train_label_fire2.npy")
    train_data, train_labels, test_data, test_labels = Train_Test_split(Train_Image,Lable_Image, fraction)
    print(len(train_data))
    CNN=CNN_Classification()
    print ("Train data shape: ", train_data.shape)
    print ("Test data shape: ", test_data.shape)
    idx = np.random.permutation(train_data.shape[0])
    ntrain = len(train_data)
    #CNN.fit(train_data[idx], train_labels[idx], batch_size = 64, step_ber_epoch = ntrain// batch_size, epochs = 10)
    CNN.fit(train_data[idx], train_labels[idx], batch_size = 64, epochs = 30)
    #Save weight
    CNN.save_weights('weightfire2.h5')
    predicted_test_labels = np.argmax(CNN.predict(test_data), axis=1)
    test_labels = np.argmax(test_labels, axis=1)
    print ("Actual test labels:", test_labels)
    print ("Predicted test labels:", predicted_test_labels)
    print ("Accuracy score:", accuracy_score(test_labels, predicted_test_labels))

2273
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 55, 55, 64)        4864      
_________________________________________________________________
batch_normalization_4 (Batch (None, 55, 55, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 128)       131200    
_________________________________________________________________
batch_normalization_5 (Batch (None, 24, 24, 128)       512       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 256) 

In [20]:
#Load Model
json_file= open('modelfire2.json','r')
loaded_model_json= json_file.read()
json_file.close()
Loaded_Model = model_from_json(loaded_model_json)
#Load weights into new model
Loaded_Model.load_weights("weightfire2.h5")
print("Loaded")

Loaded


In [0]:
def Load_Image(test_dir):
    test_images = []
    for test_path in test_dir:
      Images_Positive=os.listdir(test_path)
      for image in Images_Positive:
          path=os.path.join(test_path,image)
          img = cv2.imread(path)
          test_images.append(transform.resize(img,(224,224,3)))
    return np.array(test_images)

In [0]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        print(filename)
        if img is not None:
            images.append(img);
    return images

In [0]:
folder="/content/drive/My Drive/Fire/Testnew/NoFire_2"
Predict_Slot=[]
filename_array = []
for filename in os.listdir(folder):
    images = []
    img = cv2.imread(os.path.join(folder,filename))
    images.append(transform.resize(img, (224,224,3)))
    images= np.array(images)
    if(Loaded_Model.predict(images)[0][0]>0.7):
      Predict_Slot.append(1)
      #print("BusySlot: %s" % (filename))
      filename_array += ["fire: %s" % (filename)]

    else:
      Predict_Slot.append(0)
      #print("FreeSlot: %s" % (filename))
      filename_array += ["nofire: %s" % (filename)]
with open('Checkdata.txt', 'w') as f:
    for item in filename_array:
        f.write("%s\n" % item)

In [38]:
#Check Fire Slot
Index_Slot= []
Count=0;
for i in range(len(Predict_Slot)):
  if(Predict_Slot[i]==1):
    Count +=1;
    Index_Slot.append(i)
    print("firestt: %s" % (filename), i+1)
print("Total Number of Fire:", Count)
with open('Number fire.txt', 'w') as f:
        f.write("Total Number of Free Slot %s\n" % Count)

firestt: nofiresamp6_frame50.jpg 25
firestt: nofiresamp6_frame50.jpg 40
firestt: nofiresamp6_frame50.jpg 42
firestt: nofiresamp6_frame50.jpg 45
firestt: nofiresamp6_frame50.jpg 47
firestt: nofiresamp6_frame50.jpg 67
firestt: nofiresamp6_frame50.jpg 72
firestt: nofiresamp6_frame50.jpg 93
Total Number of Fire: 8
